# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

# Описание данных

- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.

- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [1]:
#Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Запишем и изучим значения датафрейма
df = pd.read_csv("/datasets/insurance.csv")
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
#Изучим тип данных и количество заполненных значений 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
#Приведём заголовки столбцов к нижнему и змеиному регистру
df = df.rename(columns = {'Пол': 'пол', 'Возраст': 'возраст', 'Зарплата': 'зарплата', 'Члены семьи': 'члены_семьи', 
                          'Страховые выплаты': 'страховые_выплаты'})

In [5]:
#Проверим изменения
df.head()

,пол,возраст,зарплата,члены_семьи,страховые_выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


### Вывод

С данными всё в порядке, нет пропусков, тип значений менять тоже не нужно. Заголовки были приведены к нижнему и змеиному регистру.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** $R2$ не изменится.


**Обоснование:** 
 
**ДОКАЗАТЕЛЬСТВО**

*До умножения на обратимую матрицу:*

$a = Xw$

$a = X(X^{T}X)^{-1}X^{T}y$ - для общего случая

Но, если у нас матрица $X$ - квадратная и её определитель $det$ $\ne$ 0, тогда для неё существует обратная к ней матрица $X^{-1}$, и наша формула преобразуется в:

$a = X(X^{T}X)^{-1}X^{T}y = XX^{-1}(X^{T})^{-1}X^{T}y = y$, т.к. $XX^{-1} = X^{-1}X = E$


*После умножения на обратимую матрицу:*

$a = XPw$,  где $P$ - обратимая матрица

*Тогда*

$a = XP((XP)^{T}(XP))^{-1}(XP)^{T}y = XP(P^{T}(X^{T}X)P)^{-1}(XP)^{T}y = XPP^{-1}(X^{T}X)^{-1}(P^{T})^{-1}P^{T}X^{T}y =  X(X^{T}X)^{-1}X^{T}y$ 

что совпадает с первоначальным вариантом, значит, качество линейной регрессии не изменится.

*По аналогии, если у нас матрица $X$ - квадратная и её определитель $det$ $\ne$ 0, тогда для неё существует обратная к ней матрица $X^{-1}$, и наша формула преобразуется в:*

$a = y$

                                                                                                                
                                                                                                                ч.т.д

## Алгоритм преобразования

**Алгоритм**

1. Выделим признаки и целевой признак.
2. Сгенерируем случайную матрицу 4x4 и проверим, что она обратимая.
3. Попробуем провести математические операции (сложение, вычитание, умножение, деление) со значениями нашего датасета.
4. Умножим на сгенерированную матрицу 4x4.

**Обоснование**

Считаю, что для проверки качества моделей до и после преобразования достаточно "поиграться" со значениями нашего датафрейма. Измения будут практически минимальные.

## Проверка алгоритма

In [6]:
#Выделим признаки
X = df.drop('страховые_выплаты', axis=1)
y = df['страховые_выплаты']

In [7]:
#Посмотрим форму
X.shape, y.shape

((5000, 4), (5000,))

In [8]:
#Напишем функцию для расчёта метрики качества для модели линейной регрессии
def R_2_score(x, y):
    model = LinearRegression().fit(x, y)
    prediction = model.predict(x)
    r2 = r2_score(y, prediction)
    return r2

In [15]:
#Создадим случайную матрицу 4x4 и проверим её обратимость
A = np.random.rand(4, 4)
B = np.linalg.inv(A)
print('Матрица A:')
print(A)
print('Обратная к матрице A:')
print(B)

Матрица A:
[[0.47639273 0.80964218 0.06077061 0.64103875]
 [0.53343748 0.36137606 0.01980998 0.13318678]
 [0.29236239 0.28362317 0.44342489 0.71119766]
 [0.19237033 0.35160552 0.42813262 0.494111  ]]
Обратная к матрице A:
[[-0.98075023  2.49249205  1.42867485 -1.45582326]
 [ 1.19057178 -0.45939804 -3.15395842  3.11887601]
 [-1.62048302  0.81459492 -1.59868517  4.18383781]
 [ 0.93873092 -1.34931077  3.07332518 -3.25391304]]


In [11]:
#Проверим метрику R2 до всех преобразований
R_2_score(X, y)

0.42494550286668

In [12]:
#Проверим метрику R2 с перемножением на матрицу
X_new_1 = X @ A
R_2_score(X_new_1, y)

0.42494550286668153

In [13]:
#Проверим метрику R2 с математическими операциями
X_new_2 = (((X * 10) - 15) + 7) / 2 
R_2_score(X_new_2, y)

0.42494550286668

In [14]:
#Проверим метрику R2 с математическими операциями и перемножением на матрицу
X_new_3 = X_new_2 @ A
R_2_score(X_new_3, y)

0.4249455028666965

### Вывод

Метрика R2 до всех преобразований = 0.42494550286668

Метрика R2 с перемножением на матрицу = 0.4249455028667004

Метрика R2 с математическими операциями = 0.42494550286668

Метрика R2 с математическими операциями и перемножением на матрицу = 0.42494550286667965

Как видно, изменения очень незначительные.



## Общий вывод

Таким образом, была проверена теория об умножении матриц (качество метрики не изменится), был предложен алгоритм преобразования, и всё это проверено на примерах с проведением умножения на случайную обратимую матрицу и математическими операциями над данными нашего датафрейма.

**ИТОГ: качество метрики R2 до и после преобразований почти не изменяется (можно сказать, что не изменяется).**